In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(oov_token='<nothing>')

In [2]:
file = open('Data.txt', 'r', encoding = 'utf-8')
text = file.read()
file.close()

In [3]:
tokenizer.fit_on_texts([text])
tok_len = len(tokenizer.word_index)

In [4]:
input_sequences = []
for sentences in text.split('\n'):
  tokenized_sen = tokenizer.texts_to_sequences([sentences])[0]
  for i in range(1,len(tokenized_sen)):
    input_sequences.append(tokenized_sen[:i+1])

In [5]:
max_len = max([len(x) for x in input_sequences])
max_len

231

In [6]:
from keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding='pre')

In [7]:
X = padded_input_sequences[:,:max_len-1]
y = padded_input_sequences[:,-1:]

In [8]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes = tok_len + 1)

In [9]:
from keras.layers import Embedding, Dense, LSTM
from keras.models import Sequential

In [14]:
model = Sequential()
model.add(Embedding(tok_len + 1, 200, input_length = max_len - 1))
model.add(LSTM(250))
model.add(Dense(tok_len + 1, activation = 'softmax'))

In [15]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 230, 200)          220000    
                                                                 
 lstm_1 (LSTM)               (None, 250)               451000    
                                                                 
 dense_1 (Dense)             (None, 1100)              276100    
                                                                 
Total params: 947100 (3.61 MB)
Trainable params: 947100 (3.61 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
model.fit(X, y, epochs = 100)

Epoch 1/100
115/115 [==============================] - 14s 92ms/step - loss: 6.3422 - accuracy: 0.0420
Epoch 2/100
115/115 [==============================] - 8s 67ms/step - loss: 5.8765 - accuracy: 0.0507
Epoch 3/100
115/115 [==============================] - 4s 32ms/step - loss: 5.6832 - accuracy: 0.0636
Epoch 4/100
115/115 [==============================] - 4s 37ms/step - loss: 5.4632 - accuracy: 0.0786
Epoch 5/100
115/115 [==============================] - 3s 23ms/step - loss: 5.2047 - accuracy: 0.0955
Epoch 6/100
115/115 [==============================] - 2s 21ms/step - loss: 4.9219 - accuracy: 0.1132
Epoch 7/100
115/115 [==============================] - 3s 24ms/step - loss: 4.6018 - accuracy: 0.1350
Epoch 8/100
115/115 [==============================] - 2s 19ms/step - loss: 4.2647 - accuracy: 0.1612
Epoch 9/100
115/115 [==============================] - 3s 26ms/step - loss: 3.9053 - accuracy: 0.2079
Epoch 10/100
115/115 [==============================] - 2s 20ms/step - loss: 3.53

In [21]:
model.save('The_Verdict.keras')

In [23]:
import pickle
with open('tokenizer.pickle', 'wb') as handle:
  pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)